# Projet : CryptoBot avec Binance (OPA)

## Phase 2 : Organisation des Données

### Objectif du Document:

Ce doc a pour but de présenter les différentes étapes pour : 
- Réupérer une donnée Historique, 
- Alimenter la Base Mongo, 
- Alimenter la table SQL Histo

### Step 1:  Initialisation des Modules

In [1]:
from Binance.Data import Binance_Histo as Histo
from Binance.Data import Binance_Live as live
from Binance.Dao import Drivers_MongoDB as DAO_MB
from Binance.Dao import Drivers_SQlite as DAO_SQL
from Binance.Utils import Utilitaire as util
from Binance.Utils import Technical_Analyst as util_TA

import pandas as pd

Symbol = 'ETHUSDT'
Nom_Base_SQL = "data2.db"
Nom_Base_Mongo = "OPA"

### Step 2: Chargement Fichiers Historique 

In [2]:
Liste_fichiers = list()

Donnée_Historique_URL= Histo.Binance_Histo([Symbol], ['1h'], Frequence= 'M', DateDebut = '2017-01-01')
Donnée_Historique_URL.get_ListeFichier()
Donnée_Historique_URL.TelechargeFichier()

for Monfichier in Donnée_Historique_URL.L_Fichier:
    Liste_fichiers.append(Monfichier['Nom'])

print('ci dessous la liste des fichiers à charger')
print(Liste_fichiers[0:2])


KeyboardInterrupt: 

### Step 3: Connexion à la base Mongo DB

In [3]:
# Connexion à la base avec les fichiers à charger


Donnée_Historique_Mongo = DAO_MB.Drivers_MongoDB(Liste_fichiers, Host  = 'localhost', Port = 27017, NomDB=Nom_Base_Mongo)
Donnée_Historique_Mongo.ChargeFichiers()

### Step 4:Connexion à la base SQLite

In [4]:
#Connexion à la base SQLite

 
Base_SQL = DAO_SQL.Drivers_SQLite(Nom_Base_SQL)


Database created successfully!


#### Step 4.1: Alimentation Table Dim Temps

In [5]:
#Récupération des données temps depuis la Base Mongo

Liste_Temps = list(Donnée_Historique_Mongo.DBMongo[Symbol].find({}, {'Detail.Close_time' : 1, 'Detail.Open_time': 1}))

In [6]:
 
# Formatage du Résultat dns un DataFrame

R = list()
for i in Liste_Temps :
    for y in i['Detail']:
        if y['Open_time'] not in R:
            R.append(y['Open_time'])
        
        if y['Close_time'] not in R:
            R.append(y['Close_time'])
            
DimTemps = pd.DataFrame(R, columns = ['ID_TEMPS'] , dtype='float')

DimTemps.head() 

,ID_TEMPS
0,1.502946e+12
1,1.502950e+12
2,1.502950e+12
3,1.502953e+12
4,1.502953e+12


In [7]:
# Mise à jour du DataFrame au format de la table DimTemps
# Mise à jour du DataFrame au format de la table DimTemps
 
DimTemps['SECONDES'] = DimTemps['ID_TEMPS'].apply(util.Convertir_Timestamp, formatDate=('ss'))
DimTemps['MINUTES'] = DimTemps['ID_TEMPS'].apply(util.Convertir_Timestamp, formatDate=('mm'))
DimTemps['HEURE'] = DimTemps['ID_TEMPS'].apply(util.Convertir_Timestamp, formatDate=('HH'))
DimTemps['JOUR'] = DimTemps['ID_TEMPS'].apply(util.Convertir_Timestamp, formatDate=('DD'))
DimTemps['MOIS'] = DimTemps['ID_TEMPS'].apply(util.Convertir_Timestamp, formatDate=('MM'))
DimTemps['ANNEE'] = DimTemps['ID_TEMPS'].apply(util.Convertir_Timestamp, formatDate=('YYYY'))
    
DimTemps.head(5)
 

,ID_TEMPS,SECONDES,MINUTES,HEURE,JOUR,MOIS,ANNEE
0,1.502946e+12,0,0,7,17,8,2017
1,1.502950e+12,59,59,7,17,8,2017
2,1.502950e+12,0,0,8,17,8,2017
3,1.502953e+12,59,59,8,17,8,2017
4,1.502953e+12,0,0,9,17,8,2017


In [8]:
# Alimentation Table Dim_Temps

Base_SQL.Alim_DimTemps(DimTemps)
res = Base_SQL.Select('select * from DIM_TEMPS limit 5;')
print(res)


[(1502946000000, '0.0', '0.0', '7.0', '17.0', '8.0', '2017.0', '2023-03-09'), (1502949599999, '59.0', '59.0', '7.0', '17.0', '8.0', '2017.0', '2023-03-09'), (1502949600000, '0.0', '0.0', '8.0', '17.0', '8.0', '2017.0', '2023-03-09'), (1502953199999, '59.0', '59.0', '8.0', '17.0', '8.0', '2017.0', '2023-03-09'), (1502953200000, '0.0', '0.0', '9.0', '17.0', '8.0', '2017.0', '2023-03-09')]


#### Step 4.2: Alimentation Table Dim Symbol

In [9]:

Liste_Symbole = list(Donnée_Historique_Mongo.DBMongo[Symbol].find({}, {"Symbol" : 1, "Intervalle" : 1, "_id" : 0}))

R2 = list()
for i in Liste_Symbole:
    a = {"NOM_SYMBOL" : i['Symbol'], "INTERVALLE" : i['Intervalle']} 
    if a not in R2:
        R2.append(a)
        
DimSymbol = pd.DataFrame(R2)


In [10]:


DLive = live.Binance_Live()
Info_symbol = DLive.exchange_info(Symbol)


i = {'NOM_SYMBOL' : Info_symbol['symbols'][0]['symbol'],
                              'BaseAsset' : Info_symbol['symbols'][0]['baseAsset'],
                              'QuoteAsset' : Info_symbol['symbols'][0]['quoteAsset']}
df = pd.DataFrame([i ])
DimSymbol = DimSymbol.merge(df, on = 'NOM_SYMBOL' )



In [11]:
Base_SQL.Alim_DimSymbol(DimSymbol)
res = Base_SQL.Select('select * from DIM_SYMBOL limit 5;')
print(res)
            

[(1, 'ETHUSDT', '1h', 'ETH', 'USDT', '2023-03-09')]


#### Step 4.3: Alimentation Table Fait Situation Histo

In [12]:

L = list()

for doc in Donnée_Historique_Mongo.get_AllDocuments(Symbol):
    for detail in doc['Detail']:
        L.append({'ID_TEMPS' :detail['Close_time'], 
                  'NOM_SYMBOL' : doc['Symbol'], 
                  'INTERVALLE' : doc['Intervalle'], 
                  'VALEUR_COURS' : detail['Close'] ,
                  'IND_QUOTEVOLUME' : detail['Quote_asset_volume'] ,
                  'LOW' : detail['Low']
                 })

FaiCoursHisto = pd.DataFrame(L)
FaiCoursHisto['ID_TEMPS'] = FaiCoursHisto['ID_TEMPS'].astype(float)
FaiCoursHisto['IND_QUOTEVOLUME'] = FaiCoursHisto['IND_QUOTEVOLUME'].astype(int)
FaiCoursHisto['LOW'] = FaiCoursHisto['LOW'].astype(float)
FaiCoursHisto['VALEUR_COURS'] = FaiCoursHisto['VALEUR_COURS'].astype(float)
FaiCoursHisto.head()


,ID_TEMPS,NOM_SYMBOL,INTERVALLE,VALEUR_COURS,IND_QUOTEVOLUME,LOW
0,1.502950e+12,ETHUSDT,1h,303.10,114043,300.00
1,1.502953e+12,ETHUSDT,1h,302.68,92034,301.90
2,1.502957e+12,ETHUSDT,1h,307.96,229725,302.60
3,1.502960e+12,ETHUSDT,1h,308.62,46559,307.00
4,1.502964e+12,ETHUSDT,1h,310.00,145510,308.62


In [13]:
"""
#Recherche Id Symbol à partir de la base SQL
# Simplified Select statement
res = Base_SQL.Select('SELECT ID_SYMBOL, NOM_SYMBOL, INTERVALLE FROM DIM_SYMBOL')
    
df = pd.DataFrame(res, columns=['ID_SYMBOL', 'NOM_SYMBOL', 'INTERVALLE'])

# Inner join to merge with FaiCoursHisto
FaiCoursHisto = pd.merge(FaiCoursHisto , df, on='NOM_SYMBOL', how='inner')


"""  
L = list()

res = Base_SQL.Select('select ID_SYMBOL,NOM_SYMBOL,INTERVALLE  from DIM_SYMBOL;')

for i in res:
    (a,b,c) = i
    L.append({'ID_SYMBOL' : a, 'NOM_SYMBOL' : b, 'INTERVALLE' : c})
    
df = pd.DataFrame(L)

FaiCoursHisto = FaiCoursHisto.merge(df, how = 'inner')

FaiCoursHisto.head()

,ID_TEMPS,NOM_SYMBOL,INTERVALLE,VALEUR_COURS,IND_QUOTEVOLUME,LOW,ID_SYMBOL
0,1.502950e+12,ETHUSDT,1h,303.10,114043,300.00,1
1,1.502953e+12,ETHUSDT,1h,302.68,92034,301.90,1
2,1.502957e+12,ETHUSDT,1h,307.96,229725,302.60,1
3,1.502960e+12,ETHUSDT,1h,308.62,46559,307.00,1
4,1.502964e+12,ETHUSDT,1h,310.00,145510,308.62,1


In [15]:
# Calcul des Indicateurs


trixLength = 9
trixSignal = 21

FaiCoursHisto['IND_TRIX_HISTO']=util_TA.calculate_trix_histo_ta(FaiCoursHisto['VALEUR_COURS'],9,21)
FaiCoursHisto['IND_SMA_20'] = util_TA.Calculer_SMA(FaiCoursHisto['VALEUR_COURS'], 20)
FaiCoursHisto['IND_SMA_30'] = util_TA.Calculer_SMA(FaiCoursHisto['VALEUR_COURS'], 30)
FaiCoursHisto['IND_CHANGEPERCENT'] = util_TA.Calculer_Change_Percent(FaiCoursHisto['VALEUR_COURS'])
FaiCoursHisto['IND_STOCH_RSI']=util_TA.Calculer_RSI_Stochastique_TA(FaiCoursHisto['VALEUR_COURS'], 14, 3, 3)
FaiCoursHisto['IND_RSI'] =  util_TA.Calculer_RSI(FaiCoursHisto['VALEUR_COURS'])




FaiCoursHisto = FaiCoursHisto[['ID_TEMPS',  'ID_SYMBOL','VALEUR_COURS', 'IND_SMA_20', 'IND_SMA_30', 'IND_QUOTEVOLUME', 'IND_CHANGEPERCENT', 'IND_STOCH_RSI', 'IND_RSI', 'IND_TRIX_HISTO','LOW']]


In [16]:
# Alimentation table SQL

Base_SQL.Alim_FaitSituation_Histo(FaiCoursHisto)
res = Base_SQL.Select('select * from FAIT_SIT_COURS_HIST limit 5;')
print(res)
            

[(1, 1502949599999, 1, 303.1, None, None, 114043, None, None, None, None, 300, '2023-03-09'), (2, 1502953199999, 1, 302.68, None, None, 92034, -0.13876040703053255, None, None, None, 301.9, '2023-03-09'), (3, 1502956799999, 1, 307.96, None, None, 229725, 1.7145083777113823, None, None, None, 302.6, '2023-03-09'), (4, 1502960399999, 1, 308.62, None, None, 46559, 0.21385522649213431, None, None, None, 307, '2023-03-09'), (5, 1502963999999, 1, 310, None, None, 145510, 0.4451612903225792, None, None, None, 308.62, '2023-03-09')]


CONSOMATION DES DONNEES

In [ ]:
makerFee = 0.0002
takerFee = 0.0007
capital = 1000
rapport,dfTest,dt=util_TA.Backtest_pair(capital,makerFee,takerFee,Symbol,Nom_Base_SQL)
print (rapport)
util_TA.affiche_graphe_score(dt,dfTest)